## Pandas:数据的分组和聚合

内容介绍:

数据分组汇总演示图

 ![avatar](combine.png)

In [7]:
import numpy as np
import pandas as pd
import json

In [11]:
# 示例数据
# 使用 Python JSON 模块载入数据
with open('szse_stock.json','rb') as f:
    data = json.loads(f.read())

# 展平数据
df = pd.json_normalize(data, record_path =['stockList'])
print(df.head())

         orgId category    code pinyin  zwjc
0  gssz0000001       A股  000001   payh  平安银行
1  gssz0000002       A股  000002    wka   万科A
2  gssz0000004       A股  000004   ghwa  国华网安
3  gssz0000005       A股  000005   stxy  ST星源
4  gssz0000006       A股  000006   szya  深振业A


In [12]:
df0 = pd.read_csv('directory.csv')
df0.head()

,Unnamed: 0,Unnamed: 0.1,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,AvgScore,Score,Id,Relativity
0,0,0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51,6.5,8 10 2 6,0,0
1,1,1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42,0.0,NaN,1,0
2,2,2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39,0.0,NaN,2,0
3,3,3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48,0.0,NaN,3,0
4,4,4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51,0.0,NaN,4,0


In [51]:
idx = pd.Index(['小刚','小花','小敏','小强','小明'],name='姓名')
cls = pd.Index(['语文','英语','数学'],name='科目')
df00 = pd.DataFrame(np.random.randint(50,98,size=(5,3)),index=idx,columns=cls)
df00['班级']=['二班','一班','二班','一班','一班']
df00['性别']=['男','女','男','女','男']
df00

科目,语文,英语,数学,班级,性别
姓名,,,,,
小刚,86,51,62,二班,男
小花,58,95,60,一班,女
小敏,63,67,51,二班,男
小强,89,68,66,一班,女
小明,69,93,50,一班,男


### 1.groupby()

In [52]:
#groupby()函数返回迭代对象。包含组名的元组序列
g = df00.groupby(by='班级')

In [53]:
for i in g:
    print(i)

('一班', 科目  语文  英语  数学  班级 性别
姓名                   
小花  58  95  60  一班  女
小强  89  68  66  一班  女
小明  69  93  50  一班  男)
('二班', 科目  语文  英语  数学  班级 性别
姓名                   
小刚  86  51  62  二班  男
小敏  63  67  51  二班  男)


In [69]:
for bj,da in g:
    print('组名:{}'.format(bj))
    print('*'*20)
    print(da)
    print(type(da))

组名:一班
********************
科目  语文  英语  数学  班级 性别
姓名                   
小花  58  95  60  一班  女
小强  89  68  66  一班  女
小明  69  93  50  一班  男
<class 'pandas.core.frame.DataFrame'>
组名:二班
********************
科目  语文  英语  数学  班级 性别
姓名                   
小刚  86  51  62  二班  男
小敏  63  67  51  二班  男
<class 'pandas.core.frame.DataFrame'>


In [73]:
#使用两个条件分组
g2 = df00.groupby(by=['班级','性别']).mean()
g2

科目       语文    英语    数学
班级 性别                  
一班 女   73.5  81.5  63.0
   男   69.0  93.0  50.0
二班 男   74.5  59.0  56.5

In [74]:
type(g2)

pandas.core.frame.DataFrame

### 2.聚合方法

groupby()形成的对象，本身具有计算聚合值的方法。

In [62]:
#获取班级的平均分。
g = df00.groupby(by=['班级'])
g.mean()

科目,语文,英语,数学
班级,,,
一班,72.0,85.333333,58.666667
二班,74.5,59.000000,56.500000


In [63]:
#也可以在group对象后面获取单独的列，进行计算
g['语文'].mean()

班级
一班    72.0
二班    74.5
Name: 语文, dtype: float64

In [65]:
#在分组返回值的基础上语法糖的简化写法
df00['语文'].groupby(df00['班级']).mean()

班级
一班    72.0
二班    74.5
Name: 语文, dtype: float64

In [67]:
#as_index参数的使用方法，优化显示结果的索引
df00.groupby(by=['班级'],as_index=False).mean()

科目,班级,语文,英语,数学
0,一班,72.0,85.333333,58.666667
1,二班,74.5,59.000000,56.500000


### 3.自定义聚合函数的方法???

In [68]:
#计算每个人的分数同班级平均分的差
def my_diff(arr):
    return arr.max()-arr.min()

#agg/aggregate
df00.groupby(by=['班级']).agg(my_diff)

科目,语文,英语,数学
班级,,,
一班,31,27,16
二班,23,16,11


### 4.数据处理拓展

针对星巴克门店数据的初步分析

In [87]:
g4 = df0.groupby('Country').count()
g4

,Unnamed: 0,Unnamed: 0.1,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Postcode,Phone Number,Timezone,Longitude,Latitude,AvgScore,Score,Id,Relativity
Country,,,,,,,,,,,,,,,,,,
AD,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
AE,144,144,144,144,144,144,144,144,144,24,78,144,144,144,144,0,144,144
AR,108,108,108,108,108,108,108,108,108,100,29,108,108,108,108,0,108,108
AT,18,18,18,18,18,18,18,18,18,18,17,18,18,18,18,0,18,18
AU,22,22,22,22,22,22,22,22,22,22,0,22,22,22,22,0,22,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TT,3,3,3,3,3,3,3,3,3,3,0,3,3,3,3,0,3,3
TW,394,394,394,394,394,394,394,394,394,365,39,394,394,394,394,0,394,394
US,13608,13608,13608,13608,13608,13608,13608,13608,13608,13607,13122,13608,13608,13608,13608,1,13608,13608


In [88]:
g4.iloc[:20]

,Unnamed: 0,Unnamed: 0.1,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Postcode,Phone Number,Timezone,Longitude,Latitude,AvgScore,Score,Id,Relativity
Country,,,,,,,,,,,,,,,,,,
AD,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
AE,144,144,144,144,144,144,144,144,144,24,78,144,144,144,144,0,144,144
AR,108,108,108,108,108,108,108,108,108,100,29,108,108,108,108,0,108,108
AT,18,18,18,18,18,18,18,18,18,18,17,18,18,18,18,0,18,18
AU,22,22,22,22,22,22,22,22,22,22,0,22,22,22,22,0,22,22
AW,3,3,3,3,3,3,3,3,3,0,3,3,3,3,3,0,3,3
AZ,4,4,4,4,4,4,4,4,4,3,4,4,4,4,4,0,4,4
BE,19,19,19,19,19,19,19,19,19,19,1,19,19,19,19,0,19,19
BG,5,5,5,5,5,5,5,5,5,1,0,5,5,5,5,0,5,5


In [79]:
type(g4)

pandas.core.frame.DataFrame

In [116]:
g4.loc[['US','CN']]

,Unnamed: 0,Unnamed: 0.1,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Postcode,Phone Number,Timezone,Longitude,Latitude,AvgScore,Score,Id,Relativity
Country,,,,,,,,,,,,,,,,,,
US,13608,13608,13608,13608,13608,13608,13608,13608,13608,13607,13122,13608,13608,13608,13608,1,13608,13608
CN,2734,2734,2734,2734,2734,2734,2734,2734,2734,2192,1337,2734,2734,2734,2734,1,2734,2734


In [119]:
g4.size

pandas.core.frame.DataFrame